In [32]:
import pandas as pd
from tabulate import tabulate

# Cargar los datos
reserves = pd.read_csv('https://github.com/ClaudiaPomahuallca/PC3_ClaudiaYesenia/raw/refs/heads/main/Reserves%20of%20foreign%20exchange%20and%20gold.csv')
energy = pd.read_csv('https://github.com/ClaudiaPomahuallca/PC3_ClaudiaYesenia/raw/refs/heads/main/Energy%20consumption%20per%20capita.csv')
electricity = pd.read_csv('https://github.com/ClaudiaPomahuallca/PC3_ClaudiaYesenia/raw/refs/heads/main/Electricity%20-%20installed%20generating%20capacity.csv')
education = pd.read_csv('https://github.com/ClaudiaPomahuallca/PC3_ClaudiaYesenia/raw/refs/heads/main/Education%20expenditures.csv')

# Función para procesar y limpiar datos
def process_df(df, value_col, new_name):
    df = df[['name', 'region', 'date_of_information', value_col]].copy()
    df.columns = ['country', 'region', 'date', new_name]
    df[new_name] = pd.to_numeric(
        df[new_name].astype(str).str.replace('[^\d.]', '', regex=True),
        errors='coerce'
    )
    return df.dropna(subset=[new_name])


In [33]:
# Procesar cada dataset
reserves_clean = process_df(reserves, 'value', 'reserves_usd')
energy_clean = process_df(energy, ' Btu/person', 'energy_btu_per_person')
electricity_clean = process_df(electricity, ' kW', 'electricity_kw')
education_clean = process_df(education, '% of GDP', 'education_pct_gdp')

In [34]:
# Combinar todos los datos ignorando fechas diferentes
def combine_data(*dfs):
    combined = pd.concat(dfs)
    # Para cada variable, tomar el valor más reciente no nulo por país
    combined = combined.groupby(['country', 'region']).agg({
        'reserves_usd': 'last',
        'energy_btu_per_person': 'last',
        'electricity_kw': 'last',
        'education_pct_gdp': 'last'
    }).reset_index()
    # Eliminar países que no tienen los 4 valores completos
    combined = combined.dropna()
    return combined

final_df = combine_data(reserves_clean, energy_clean, electricity_clean, education_clean)

In [35]:
# Configuración para mostrar toda la tabla
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Mostrar tabla formateada
print(" DATASET FINAL CONSOLIDADO POR PAÍS")
print(tabulate(
    final_df,
    headers='keys',
    tablefmt='psql',
    showindex=False,
    floatfmt=".1f"
))

# Guardar a CSV
final_df.to_csv('dataset_final_consolidado.csv', index=False)
print("\n Datos guardados en 'dataset_final_consolidado.csv'")

 DATASET FINAL CONSOLIDADO POR PAÍS
+-----------------------------------+-----------------------------------+-----------------+-------------------------+------------------+---------------------+
| country                           | region                            |    reserves_usd |   energy_btu_per_person |   electricity_kw |   education_pct_gdp |
|-----------------------------------+-----------------------------------+-----------------+-------------------------+------------------+---------------------|
| Afghanistan                       | South Asia                        |    9749000000.0 |               3380000.0 |         627000.0 |                 2.9 |
| Albania                           | Europe                            |    6455000000.0 |              27407000.0 |        2857000.0 |                 3.1 |
| Algeria                           | Africa                            |   68448000000.0 |              61843000.0 |       22591000.0 |                 7.0 |
| Angola  

In [36]:
# Calcular estadísticos por región
stats = final_df.groupby('region').agg({
    'reserves_usd': ['mean', 'min', 'max'],
    'energy_btu_per_person': ['mean', 'min', 'max'],
    'electricity_kw': ['mean', 'min', 'max'],
    'education_pct_gdp': ['mean', 'min', 'max']
}).round(2)


In [37]:
# Renombrar columnas para mejor legibilidad
stats.columns = ['_'.join(col).strip() for col in stats.columns.values]
stats = stats.reset_index()

In [38]:
# Mostrar estadísticos con formato de tabla
print("\n ESTADÍSTICOS POR REGIÓN:")
print(tabulate(
    stats,
    headers='keys',
    tablefmt='psql',
    showindex=False,
    floatfmt=".2f"
))

# Guardar estadísticos a CSV
stats.to_csv('estadisticos_por_region.csv', index=False)
print("\n Estadísticos guardados en 'estadisticos_por_region.csv'")


 ESTADÍSTICOS POR REGIÓN:
+-----------------------------------+---------------------+--------------------+--------------------+------------------------------+-----------------------------+-----------------------------+-----------------------+----------------------+----------------------+--------------------------+-------------------------+-------------------------+
| region                            |   reserves_usd_mean |   reserves_usd_min |   reserves_usd_max |   energy_btu_per_person_mean |   energy_btu_per_person_min |   energy_btu_per_person_max |   electricity_kw_mean |   electricity_kw_min |   electricity_kw_max |   education_pct_gdp_mean |   education_pct_gdp_min |   education_pct_gdp_max |
|-----------------------------------+---------------------+--------------------+--------------------+------------------------------+-----------------------------+-----------------------------+-----------------------+----------------------+----------------------+--------------------------+

In [39]:
# Verificar qué columnas tenemos actualmente
print("Columnas existentes:", final_df.columns.tolist())

# Si las columnas de categorías no existen, las creamos
if 'reserves_usd_categoria' not in final_df.columns:
    # Definir función para crear categorías
    def crear_categorias(df, columna):
        try:
            return pd.qcut(
                df[columna],
                q=5,
                labels=['Muy bajo', 'Bajo', 'Medio', 'Alto', 'Muy alto'],
                duplicates='drop'
            )
        except:
            return pd.cut(
                df[columna],
                bins=5,
                labels=['Muy bajo', 'Bajo', 'Medio', 'Alto', 'Muy alto']
            )

    # Aplicar a cada variable
    final_df['reserves_usd_categoria'] = crear_categorias(final_df, 'reserves_usd')
    final_df['energy_btu_per_person_categoria'] = crear_categorias(final_df, 'energy_btu_per_person')
    final_df['electricity_kw_categoria'] = crear_categorias(final_df, 'electricity_kw')
    final_df['education_pct_gdp_categoria'] = crear_categorias(final_df, 'education_pct_gdp')


Columnas existentes: ['country', 'region', 'reserves_usd', 'energy_btu_per_person', 'electricity_kw', 'education_pct_gdp']


In [40]:
# Ahora reorganizar las columnas
column_order = ['country', 'region']
for var in ['reserves_usd', 'energy_btu_per_person', 'electricity_kw', 'education_pct_gdp']:
    column_order.extend([var, f'{var}_categoria'])


In [41]:
# Crear el DataFrame de visualización
final_df_display = final_df[column_order]

In [43]:
# Mostrar el dataset completo
print("\n DATASET COMPLETO CON CATEGORÍAS:")
print(tabulate(
    final_df_display,
    headers='keys',
    tablefmt='psql',
    showindex=False,
    floatfmt=".1f"
))

# Guardar el dataset
final_df_display.to_csv('dataset_final_con_categorias.csv', index=False)
print("\n Dataset guardado en 'dataset_final_con_categorias.csv'")


 DATASET COMPLETO CON CATEGORÍAS:
+-----------------------------------+-----------------------------------+-----------------+--------------------------+-------------------------+-----------------------------------+------------------+----------------------------+---------------------+-------------------------------+
| country                           | region                            |    reserves_usd | reserves_usd_categoria   |   energy_btu_per_person | energy_btu_per_person_categoria   |   electricity_kw | electricity_kw_categoria   |   education_pct_gdp | education_pct_gdp_categoria   |
|-----------------------------------+-----------------------------------+-----------------+--------------------------+-------------------------+-----------------------------------+------------------+----------------------------+---------------------+-------------------------------|
| Afghanistan                       | South Asia                        |    9749000000.0 | Medio                   

# Nueva sección